# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (сопоставимые или большие по объему). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг <start>  
    - еще одна матрица не нужна, можно по строкам хронить биграмы, а по колонкам униграммы  
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так). 

In [46]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML 
from collections import Counter

In [47]:
# !!! двач не самое приятное место, большое количество текстов в этом корпусе токсичные
dvach = open('2ch_corpus.txt', encoding = 'utf8').read()
news = open('lenta.txt', encoding = 'utf8').read()

In [48]:
print("Длина 1 -", len(dvach))
print("Длина 2 -", len(news))

Длина 1 - 11638405
Длина 2 - 11536552


In [49]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


In [50]:
sentence_news = [['<start>', '<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news)][:-50]
sentence_test = [['<start>', '<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news)][-50:]

In [51]:
from nltk.tokenize import sent_tokenize


def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [52]:
unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))
    trigrams_news.update(ngrammer(sentence, n=3))

In [53]:
import scipy 

matrix_news = scipy.sparse.lil_matrix((len(bigrams_news), len(unigrams_news)))

id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}

id2bigram_news = list(bigrams_news)
bigram2id_news = {word:i for i, word in enumerate(id2bigram_news)}

# вероятность расчитываем точно также
for ngram in trigrams_news:
    word1, word2, word3 = ngram.split()
    bigram = word1 + " " + word2
    trigram = word1 + " " + word2 + " " + word3
    
    matrix_news[bigram2id_news[bigram], word2id_news[word3]] =  (trigrams_news[trigram] / bigrams_news[bigram])

In [54]:

def generate(matrix, id2word, word2id, id2bigram, bigram2id, n=100, start='<start> <start>'):
    text = []
    current_idx = bigram2id[start]
    prev = start
    for i in range(n):
        prev = prev.split()[1]

        chosen = np.random.choice(list(range(matrix.shape[1])), p=matrix[current_idx].toarray()[0])
        text.append(id2word[chosen])
        if id2word[chosen] == '<end>':
            chosen = word2id['<start>']
            prev = "<start> <start>"
        else:
            prev = prev + " " + id2word[chosen]
        
        current_idx = bigram2id[prev]
    
    return ' '.join(text)

In [55]:
print(generate(matrix_news, id2word_news, word2id_news, id2bigram_news, bigram2id_news).replace('<end>', '\n'))

тип и происхождение мины установлены это минометная мина с оперением времен великой отечественной войны 
 кеннеди на мысе канаверал осталось всего 102 добровольца остальные эвакуированы 
 в настоящее время анализируется 
 генерал особо подчеркнул что решение направить делегацию fbi в российскуюстолицу это острая реакция правительства сша на происходящиесобытия в москве с 6 сентября поисковые работы 
 арестованы 9 человек которые подозреваются в получении взяток и злоупотреблении общественными фондами во время налета на югославию 
 кроме прейскера нашими войсками взят в плен 
 высокая активность добровольцев позволила специалистам в короткий срок создать минимальный неприкосновенный резерв донорской крови и ее гражданам полной


In [56]:
def perplexity(probas):
    p = np.exp(np.sum(probas))
    N = len(probas)
    
    return p**(-1/N) 

Перплексия, усреднённая по 10-ти отложенным предложениям:

In [59]:
perplexities = []
for sentence in sentence_test[:10]:
    prob = []
    
    for ngram in ngrammer(sentence, n=3):
        word1, word2, word3 = ngram.split()

        gram2 = word1 + ' ' + word2
        if gram2 in bigrams_news and ngram in trigrams_news:
            prob.append(np.log(trigrams_news[ngram] / bigrams_news[gram2]))
        else:
            prob.append(np.log(0.00001))
    
    perplexities.append(perplexity(prob))


In [60]:
perplexities

[34172.911477570065,
 22587.82763143729,
 14046.609087247294,
 99999.9999999999,
 62630.64674967313,
 99999.9999999999,
 84530.6512373702,
 65691.42386718343,
 99999.99999999991,
 20890.3675130843]

In [62]:
f'mean perplexity: {np.mean(perplexities)}'

'mean perplexity: 60455.04375635654'

## Задание № 2* (2 балла). 

Прочитайте главу про языковое моделирование в книге Журафски и Мартина - https://web.stanford.edu/~jurafsky/slp3/3.pdf

Развернуто (в пределах 1000 знаков) ответьте на вопросы (по-русски):

1. Что можно делать с проблемой несловарных слов? В семинаре мы просто использовали какое-то маленькое значение вероятности, а какие есть другие способы?

Несловарные (OOV) слова – это слова, которых нет в обучающих данных, но они присутствуют в тестовом наборе. В системе с открытым словарем (open vocabulary) мы моделируем эти потенциально OOV-слова на тестовом наборе, добавляя псевдослово $<UNK>$. 
В главе утверждается, что существует два способа работы с $<UNK>$:
1.	Выбрать заранее фиксированный словарь, преобразовать любое OOV-слово в токен неизвестного слова $<UNK>$ на трейне на этапе нормализации текста, вычислить вероятности для $<UNK>$ по его встречаемости, как и для любого другого обычного слова на трейне.
2.	Если нет заранее составленного словаря нужно его неявно создать, заменяя слова в обучающих данных на $<UNK>$ на основе их частоты. 


2. Что такое сглаживание (smoothing)?

Бывает так, что слово есть в словаре, но оно появляется в незнакомом контексте в тестовом наборе. Сглаживание (smoothing или discounting) - процесс, когда таким событиям присваивается не нулевая вероятность, а какая-то часть вероятности, которая берётся у более частотных событий.